## 入门

### 读取图像
np.array = imread(img_name:str,flag:int)  
其中，flag=0表示以灰度模式读取图片，flag=1表示以BGR的默式读取图片，也是默认的模式,还有其他不常用的模式。

In [1]:
import cv2

In [2]:
img1 = cv2.imread('img1.jpeg',0)
img2 = cv2.imread('img1.jpeg',1)

### 显示图像
None = cv2.imshow(window_name:str,img:np.array)
其中，window_name是图片窗口的名称，可以随意命名，但是单独调用这个命令的使用，虽然能弹出图片的显示窗口，但是窗口显示的是灰色的纯色，没有内容。需要调用另外一个方法，让程序暂停运行才能正常显示。  

keyboard_ascii_code = cv2.waitKey(wait_seconds:int)  
其中，wait_seconds是程序等待按键响应的时间，在这个时间内按下任意键盘上的按键，将返回这个按键对应的ascii编码，超出这个时间程序返回-1这个整数值。如果不提供这个参数的话，程序会无限期的等待一个按键被按下，然后再执行后面的语句。  
一般在显示图像后面还需要跟关闭显示窗口的命令，因为imshow()只负责弹出窗口，不负责关闭窗口。  

None = cv2.namedWindow(window_name:str)  
None = cv2.destroyWindow(window_name:str)  
None = cv2.destroyAllWindows()  
这三个命令的作用分别是：创建一个有名字的窗口，关闭一个有名字的窗口，关闭所有的窗口。一般用第二个和第三个就行了

In [ ]:
#运行这段代码不能正常显示图片,只能显示灰色的窗口
cv2.imshow('test1',img1)
cv2.imshow('test2',img2)

In [3]:
#这段代码可以正常显示窗口，
#但是窗口不会弹出，需要自己从桌面下方的状态栏点出来
#然后按一下键盘上任何一个按键会关掉一个窗口
cv2.imshow('test1',img1)
cv2.imshow('test2',img2)
cv2.waitKey()
cv2.destroyWindow('test1')
cv2.waitKey()
cv2.destroyWindow('test2')

### 保存图像
True/False = cv2.imwrite(file_path:str,img:np.array)   
如果成功写入就返回True，写入失败就返回False

In [4]:
cv2.imwrite('img2.jpeg',img1)

True

## 图像处理基础

### 图像逻辑
1、读取出的图像本质上是numpy数据，根据不同的色彩模式，每个位置上的数字的取值范围不同 
2、使用imread()函数返回的图像，默认使用BGR模式，注意，这个BGR和常规的图片RGB显示模式是相反的。  
3、彩色图像的默认shape是（H，W，C），其中H是图片高度，W是图片宽度，C是图片的通道数量，一般是3个，按照BGR的顺序排列。  
4、图片的size指的是图像对应的numpy数组的所有元素的和  
5、BGR图像的数据类型一般为'uint8'，也就是8位长的整型，这是因为BGR模式下每个像素点的取值范围是（0，255），还有一些其他的图像类型和图像模式，像素点的取值范围会超过255，这个时候，数据类型就会发生变化，有些函数在进行图片处理的时候图片的数据类型或者和图片进行计算的数据的数据类型不匹配的话，会报错。

In [5]:
img2.shape,img2.size

((500, 800, 3), 1200000)

In [6]:
img1.dtype,img2.dtype

(dtype('uint8'), dtype('uint8'))

### 图像像素值操作
因为图像本质上是numpy数据，对图像的像素进行操作，等同于对numpy数组的元素进行操作，通常使用索引进行读取和修改

In [7]:
#将图片的0-99像素高度的范围变成白色
import numpy as np
tmp_array = img2[:100]
img2[:100] = 255
cv2.imshow('test',img2)
cv2.waitKey()
cv2.destroyWindow('test')
img2[:100] = tmp_array

In [8]:
#分别显示图片的，B、G、R三个不同通道，以及原图的样子
import numpy as np

img2_only_channelB = np.array(img2,copy=True)
img2_only_channelG = np.array(img2,copy=True)
img2_only_channelR = np.array(img2,copy=True)

img2_only_channelB[...,1:3] = 0
img2_only_channelG[...,0:3:1] = 0
img2_only_channelR[...,0:2] = 0

cv2.imshow('raw image',img2)
cv2.imshow('channel B',img2_only_channelB)
cv2.imshow('channel G',img2_only_channelG)
cv2.imshow('channel R',img2_only_channelR)

cv2.waitKey()
cv2.destroyAllWindows()

### 图像通道操作
除了按照一个个像素进行操作之外，cv2还有一个split函数，可以将一张图片按照通道的顺序，切分成多个单通道数据：  
tuple = cv2.split(img:np.array)  

split函数的逆向函数是merge函数，可以把多个相同尺寸的单通道图片组合成一个多通道图片:  
np.array = cv2.merge(tuple)

In [9]:
B,G,R = cv2.split(img2)
cv2.imshow('channel B',B)
cv2.imshow('channel G',G)
cv2.imshow('channel R',R)
cv2.waitKey()
cv2.destroyAllWindows()
tmp_img3 = cv2.merge((G,B,R))
cv2.imshow('merge test',tmp_img3)
cv2.waitKey()
cv2.destroyAllWindows()

## 图像修改

### 图像相加和图像叠加
图像实际上是numpy数组，所以可以直接对图片使用'+'和'+='操作来改变数组的数值，除此之外cv2还有一个函数可以进行加法运算，这个函数和直接调用+号的区别是，+号运算在数值超过当前图像模式的取值范围后会取模，而add()函数会将数值保持在最大值不变。举例来说，一个像素点的值是200，现在要加上100，如果用'+'来操作，最后得到的结果是300%255=44，而使用.add()方法，得到的结果是255，因为200+100>255，所以取255作为结果。

In [10]:
#需要注意的是，这两个操作结果的差别跟数据类型直接相关，
#如果去掉下面代码中的'dtype=np.uint8'，两种运算结果将相同
tmp_a = np.array([200],dtype=np.uint8)
tmp_b = np.array([100],dtype=np.uint8)
print('a+b:',tmp_a+tmp_b)
print('cv2.add(a,b):',cv2.add(tmp_a,tmp_b))

a+b: [44]
cv2.add(a,b): [[255]]


基于图像本质上是数组所以可以叠加的原理，cv2有一个自带的函数可以对两张图片进行叠加处理。  
np.array = cv2.addWeighted(img1,weight1,img2,weight2,gamma)。  
img1和img2要求形状和尺寸一致，计算原理如下：   
$out=img1*weight1+img2*weight2+gamma$

In [11]:
tmp_img = cv2.flip(img2,-1)
tmp_img1 = cv2.addWeighted(tmp_img,0.2,img2,0.2,0)
cv2.imshow('test',tmp_img1)
cv2.waitKey()
cv2.destroyWindow('test')

### 图像位运算和位平面分解

图像的位运算是个比较有意思的操作。默认的BGR模式像素点单个通道的最大值是255，相当于8位数据，可以把两个像素点单独拿出来比对他们的8位数据之间的关系，并进行运算，可以进行的运算包括:  

np.array = cv2.bitwise_and(img1:np.array,img2:np.array,out_img_size:tuple)  

np.array = cv2.bitwise_or(img1:np.array,img2:np.array,out_img_size:tuple)  

np.array = cv2.bitwise_not(img1:np.array,out_img_size:tuple)  

np.array = cv2.bitwise_xor(img1:np.array,img2:np.array,out_img_size:tuple)  

位运算的可以用来做一些比较特殊的操作，例如图像加密，或者是图像处理。下面是一个图像处理的例子，把一张图片用不同的位来表达

In [12]:
import numpy as np
H,W,C = img2.shape
bit_st = np.zeros((H,W,C,8),dtype=np.uint8)
out_data = np.zeros((H,W,C,8),dtype=np.uint8)
for i in range(8):
    bit_st[...,i] = 2**i #创建可以和原图像进行计算的矩阵，其中2**0表示00000001，2**1表示00000011，可以很方便的进行位运算
    out_data[...,i] = cv2.bitwise_and(img2,bit_st[...,i],(H,W,C))
    cv2.imshow(f'bit-wise{i}',out_data[...,i])
cv2.waitKey()
cv2.destroyAllWindows()

### 图像蒙版
图像蒙版可以单独使用，也可以作为参数之一传入很多函数中使用。蒙版是只由数值0和f非0值组合而成的矩阵，图片和蒙版的长宽形状应该相同，经过蒙版之后，图片在蒙版数值为0的区域显示黑色，图片在蒙版数值为非0的区域显示原有图像内容。  
蒙版的原理是乘法，即将图片和蒙版对应的numpy数组相乘，0和任何值相乘结果为0，非0值都被处理成了1，而1和任何值相乘还是任何值本身，而0在BGR中代表的颜色是黑色，所以达到了蒙版的效果。

In [13]:
#下面是蒙版作为参数传给其他函数的情况
import numpy as np
H,W,C = img2.shape
mask = np.random.randint(0,2,(H,W),dtype=np.uint8) #对于BGR图片来说，mask只能是2维的
cv2.imshow('test1',img2)
cv2.waitKey()
cv2.destroyWindow('test1')
tmp_img2 = cv2.add(img2,img2,(H,W,C),mask)
cv2.imshow('test2',tmp_img2)
cv2.waitKey()
cv2.destroyWindow('test2')

### 图像加密和解密
加密和解密主要通过bit-wise的xor运算来实现。首先虚拟出一张照片，作为密文照片，待加密的照片通过与密文照片进行逐位的xor运算可以得到加密后的照片。需要解密时，将加密后的照片与密文照片做bit-wise xor运算，就可以得到原照片。

### 图像色彩模式转换
图片有很多色彩模式，cv2中有一个类叫做color,里面包含了很多不同的色彩模式属性，再结合cvtColor()函数，可以实现图片色彩模式的转换  
np.array = cv2.cvtColor(img:np.array,color_code)  
其中color_code指的是上面说的cv2自带的各种色彩模式

In [14]:
#将默认的BGR图片转换成RGB图片
tmp_img3 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)
cv2.imshow('test',tmp_img3)
cv2.waitKey()
cv2.destroyAllWindows()

### 提取指定颜色
HSV模式下，H（Hue）代表色相，S(Saturation)代表饱和度，V(Value)代表亮度，对于某些颜色，可以在HSV色彩模式下筛选出来。具体的做法是，使用inRange()函数，根据提供的值域返回对应的图面范围，如果需要色相、饱和度、亮度同时达到对应的条件，可以先分别对这三个通道进行筛选，生成三个mask，然后对三个mask求并集，最终获得需要的范围。  

np.array = cv2.inRange(img:array,lowwer_num:int,higher_num:int)  
其中，返回的np.array内只包含两个值，一个是0，一个是255，在（lowwer_num,higher_num）范围内返回的是255，在这个范围之外的返回的是0

In [15]:
tmp_img4 = np.copy(img2)
tmp_img4 = cv2.cvtColor(tmp_img4,cv2.COLOR_BGR2HSV)
H,S,V = cv2.split(tmp_img4)
H_mask = cv2.inRange(H,0,10)
S_mask = cv2.inRange(S,10,200)
mask = H_mask & S_mask
new_img = cv2.bitwise_and(img2,img2,mask=mask)
cv2.imshow('test',new_img)
cv2.waitKey()
cv2.destroyAllWindows()

## 图像几何变换

### 调整尺寸
resize改变图片尺寸和总共的像素，需要注意的是，输出的尺寸是以宽度在前，高度在后的形式传入的。  

In [16]:
width = 300
height = 100
tmp_img5 = cv2.resize(img2,(width,height))
cv2.imshow('test',tmp_img5)
cv2.waitKey()
cv2.destroyAllWindows()


除了直接传入最后输出的尺寸，也可以采用比例的形式控制输出的长宽

In [17]:
fx = 0.5
fy = 0.5
tmp_img5 = cv2.resize(img2,None,fx=fx,fy=fy)
cv2.imshow('test',tmp_img5)
cv2.waitKey()
cv2.destroyAllWindows()

### 翻转
可以通过指定flag_code来根据不同的轴来进行翻转:  
1是水平翻转，0是垂直翻转，-1是水平垂直同时翻转

In [18]:
tmp_img8 = cv2.flip(img2,-1)
tmp_img7 = cv2.flip(img2,0)
tmp_img6 = cv2.flip(img2,1)

cv2.imshow('raw',img2)
cv2.waitKey()
cv2.destroyAllWindows()

cv2.imshow('flip_code 1',tmp_img6)
cv2.waitKey()
cv2.destroyAllWindows()

cv2.imshow('flip_code 0',tmp_img7)
cv2.waitKey()
cv2.destroyAllWindows()

cv2.imshow('flip_code -1',tmp_img8)
cv2.waitKey()
cv2.destroyAllWindows()

### 仿射（线性变换）
放射的作用是把一张方形的图片变形成一个任意的平行四边形，实现的方式是通过一个自定的M矩阵，将原坐标V向量（横坐标为x1,纵坐标为y1）通过M * V 转化得到新的坐标（x2,y2）。  
其中，自定义的矩阵M可以通过一些内置的函数求得。如：  

**求产生旋转效果的自定义矩阵**  

np.array = cv2.getRotationMatrix2D(center,angle,scale)  

返回能产生对应旋转效果的matrix。  

In [19]:
cv2.imshow('test',img2)
cv2.waitKey()
cv2.destroyAllWindows()

H,W,C = img2.shape
r_matrix = cv2.getRotationMatrix2D((W/2,H/2),45,0.5)  #注意，中点坐标是按照（width,height）的形式来传入的
tmp_img9 = cv2.warpAffine(img2,r_matrix,(W,H))  #注意，dsize是按照（W，H）的形状输入的
cv2.imshow('test',tmp_img9)
cv2.waitKey()
cv2.destroyAllWindows()

**求能进行平行转换的矩阵**  

np.array = cv2.getAffineTransform(raw_point:np.array,transformed_point:np.array)  

能通过输入原始图片的三个点形成的数组，和转换完之后的图片上对应位置三个点坐标形成的数组，来返回能产生对应变化的matrix。  
需要注意的是点的坐标计算方式，坐标轴是以图片左上角为原点，原点往右为X轴正方向，原点往下为Y轴正方向。  
需要明确的三个点为转换前后的左上、右上、左下三个点

In [20]:
cv2.imshow('test',img2)
cv2.waitKey()
cv2.destroyAllWindows()

H,W,C = img2.shape
raw_point_set = np.array([[0,0],[W-1,0],[0,H-1]],dtype=np.float32)  #注意，坐标点要按照float的数据形式创建
transformed_point_set = np.array([[0,0.5*H],[0.5*W,0],[0.5*W,H-1]],dtype=np.float32)
affine_matrix = cv2.getAffineTransform(raw_point_set,transformed_point_set)
tmp_img10 = cv2.warpAffine(img2,affine_matrix,(W,H))  #注意，dsize是按照（W，H）的形状输入的
cv2.imshow('test',tmp_img10)
cv2.waitKey()
cv2.destroyAllWindows()

**求能产生透视变换的矩阵**  

np.array = cv2.getPerspectiveTransform(raw_point_set,transformed_point_set)  

其中，raw_point_set和transformed_point_set都是一个包含四个点的矩阵，四个点分别位于图像的左上，右上，左下，右下位置

In [21]:
cv2.imshow('test',img2)
cv2.waitKey()
cv2.destroyAllWindows()

H,W,C = img2.shape
raw_point_set = np.array([[0,0],[W-1,0],[0,H-1],[W-1,H-1]],dtype=np.float32)  #注意，坐标点要按照float的数据形式创建
transformed_point_set = np.array([[0,0.8*H],[0.5*W,0],[0.2*W,H-1],[W-1,H-1]],dtype=np.float32)
pers_matrix = cv2.getPerspectiveTransform(raw_point_set,transformed_point_set)
tmp_img11 = cv2.warpPerspective(img2,pers_matrix,(W,H))  #注意，dsize是按照（W，H）的形状输入的
cv2.imshow('test',tmp_img11)
cv2.waitKey()
cv2.destroyAllWindows()

### 映射
映射的功能很强大，可以实现复制、翻转，缩放等功能，以及其他自定义的图像处理功能

return_img:np.array = remap(img:np.array, mapx:np.array, mapy:np.array)  

函数的原理如下，img像是一个字典，而mapx是和mapy产生了X坐标索引和Y坐标索引，通过产生的（X，Y）坐标，在img里面取到对应位置的值，并将这个值赋予给return_img的（X，Y）坐标里的值，**以复制一张图片作为例子，具体原理如下：**

In [22]:
cols = 4
rows = 3
i_img = np.random.randint(0,256,(rows,cols),dtype=np.uint8)
mapx = np.zeros((rows,cols),dtype=np.float32)
mapy = np.zeros((rows,cols),dtype=np.float32)
for i in range(rows):
    for j in range(cols):
        mapx.itemset((i,j),j)
        mapy.itemset((i,j),i)
o_img = cv2.remap(i_img,mapx,mapy,cv2.INTER_LINEAR)
print('i_img:\n',i_img)
print('mapx:\n',mapx)
print('mapy:\n',mapy)
print('o_img:\n',o_img)

i_img:
 [[ 67 107 249 239]
 [123  23  93  53]
 [236  78 235 224]]
mapx:
 [[0. 1. 2. 3.]
 [0. 1. 2. 3.]
 [0. 1. 2. 3.]]
mapy:
 [[0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [2. 2. 2. 2.]]
o_img:
 [[ 67 107 249 239]
 [123  23  93  53]
 [236  78 235 224]]


**使用remap对图片进行水平翻转**

In [23]:
cv2.imshow('test',img2)
cv2.waitKey()
cv2.destroyAllWindows()

H,W,C = img2.shape
mapx = np.zeros((H,W),dtype=np.float32)
mapy = np.zeros((H,W),dtype=np.float32)
for i in range(H):
    for j in range(W):
        mapx.itemset((i,j),W-j)
        mapy.itemset((i,j),i)
tmp_img12 = cv2.remap(img2,mapx,mapy,cv2.INTER_LINEAR)
cv2.imshow('test',tmp_img12)
cv2.waitKey()
cv2.destroyAllWindows()